# Building the design matrix (DARTEL)

In [3]:
%run /home/grg/git/alfa/nilearn-helper.py
%matplotlib inline

axondir = '/home/grg/data/ALFA_DWI'
spmdir = '/home/grg/spm/'

designmat_fp = osp.join(spmdir, 'designmatrix.xls')
subjects = json.load(open(osp.join(spmdir, 'data', 'subjects_dartel2.json')))
print len(subjects)

/home/grg/jupyter/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


500


In [4]:
def collect_images(spmdir, subjects, verbose=True):
    im_types = ['MD', 'FA', 'RD', 'Jacobians', 'L1', 'MD_pred', 'MD_corr']

    images = {}
    for s in subjects:
        for i, each in enumerate(im_types):
            item = glob(osp.join(spmdir, each, '*%s*.nii'%s))[0]
            images.setdefault(each, {}).update({s:item})
    return images

def getdict(data, key, column, column2=None, value=None):
    import string
    if not value is None:
        data = data[data[column2] == value]
        
    d1 = dict([(string.atoi(str(int(e))), v) for e, v in data[[key, column]].to_dict(orient='split')['data']])
    return d1

d = getdict(vvol, 'subject', 'Volume_mm3', 'StructName', 'Right-Lateral-Ventricle')
d

NameError: name 'vvol' is not defined

In [29]:
# Reading main tables with metadata
demo_fp = osp.join(spmdir, 'data', 'Data_VBM_Alfa.csv')
subj_fp= osp.join(spmdir, 'data', 'subjects_dartel2.json')
gend_fp= osp.join(spmdir, 'data', 'genders.json')
educ_fp = osp.join(spmdir, 'data', 'educyears.json')
tiv_fp = osp.join(spmdir, 'data', 'tiv.json')
fsvol_fp = osp.join(spmdir, 'data', 'aseg FS ALFA.xlsx')
ages_fp = osp.join(spmdir, 'data', 'age.json') # contains ages in days
jdg_fp = osp.join(spmdir, 'data', 'Ventricular volumes.xlsx')

demo = pd.read_csv(demo_fp)
subjects = json.load(open(subj_fp))
images = collect_images(spmdir=spmdir, subjects=subjects)

cov = pd.read_excel('/home/grg/spm/data/covariates.xls')
apo = getdict(cov, 'subject', 'apo')

In [33]:
data = []

for s in subjects:
    print s    
    #im = [images[e][s] for e in im_types]
    row = [s]
    age = ages[str(s)] / 365.25
    #educyear = educyears[str(s)]
    group = [int(apo[string.atoi(s)]==i) for i in xrange(5)]
    #gender = genders[str(s)]
    #fsvol_l = (vvol_l[s] + vvol_li[s]) / tivs[s]
    #fsvol_r = (vvol_r[s] + vvol_ri[s]) / tivs[s]
    #fsvol = fsvol_l + fsvol_r
    #jdgvol = jdg_vvol[s] / tivs[s]
    
    
    # Adding the row to the table
    #row.extend(im)
    #row.extend(group)
    row.extend([age*e for e in group])
    #row.extend([age*age*e for e in group])
    #row.extend([gender, educyear, fsvol_l, fsvol_r, fsvol, jdgvol]) #, vvol_l[s], tivs[s]])
    data.append(row)
    print row

10070
[u'10070', 50.78165639972622, 0.0, 0.0, 0.0, 0.0]
10102
[u'10102', 64.06570841889118, 0.0, 0.0, 0.0, 0.0]
10108
[u'10108', 58.68309377138946, 0.0, 0.0, 0.0, 0.0]
10235
[u'10235', 61.24572210814511, 0.0, 0.0, 0.0, 0.0]
10365
[u'10365', 48.720054757015745, 0.0, 0.0, 0.0, 0.0]
10419
[u'10419', 50.90485968514716, 0.0, 0.0, 0.0, 0.0]
10463
[u'10463', 58.41204654346338, 0.0, 0.0, 0.0, 0.0]
10530
[u'10530', 56.435318275154, 0.0, 0.0, 0.0, 0.0]
10563
[u'10563', 50.836413415468854, 0.0, 0.0, 0.0, 0.0]
10576
[u'10576', 47.21697467488022, 0.0, 0.0, 0.0, 0.0]
10630
[u'10630', 53.40177960301163, 0.0, 0.0, 0.0, 0.0]
10668
[u'10668', 57.921971252566735, 0.0, 0.0, 0.0, 0.0]
10692
[u'10692', 64.59411362080766, 0.0, 0.0, 0.0, 0.0]
10693
[u'10693', 66.09719370294319, 0.0, 0.0, 0.0, 0.0]
10703
[u'10703', 48.46543463381246, 0.0, 0.0, 0.0, 0.0]
10725
[u'10725', 56.74469541409993, 0.0, 0.0, 0.0, 0.0]
10737
[u'10737', 49.18001368925393, 0.0, 0.0, 0.0, 0.0]
10756
[u'10756', 52.41889117043121, 0.0, 0.0, 0

In [81]:
# Saving to Excel files
for each in im_types:
    print each
    columns1 = [each, 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears']
    df1 = pd.DataFrame(df, columns=columns1)
    dmat_fp = osp.join(spmdir, 'designmat', 'dm_%s_noventvol.xls'%each)
    df1.to_excel(dmat_fp, index=False)
    
    # With ventricles from FS
    columns1 = [each, 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears', 'ventricles_FS']
    df1 = pd.DataFrame(df, columns=columns1)
    dmat_fp = osp.join(spmdir, 'designmat', 'dm_%s_fsvol.xls'%each)
    df1.to_excel(dmat_fp, index=False)
    
    # With ventricles from JDG
    columns1 = [each, 
       'Apoe2-3', 'Apoe2-4', 'Apoe3-3', 'Apoe3-4', 'Apoe4-4', 
       'age23', 'age24', 'age33', 'age34', 'age44', 
       'agesq23', 'agesq24', 'agesq33', 'agesq34', 'agesq44', 
       'gender', 'educyears', 'ventricles_JDG']
    df1 = pd.DataFrame(df, columns=columns1)
    dmat_fp = osp.join(spmdir, 'designmat', 'dm_%s_jdgvol.xls'%each)
    df1.to_excel(dmat_fp, index=False)

MD
FA
RD
Jacobians
L1
MD_pred
MD_corr


Analysis ready to be run.

In [9]:
import sys
sys.path.append('/home/grg/git/alfa/')

In [10]:
import multireg_spm12 as mreg

excel_fp = '/home/grg/spm/data/designmatrix2.xls' # table containing model data
paramtype = 'MD' # given column from the previous table
analysis_wd = '/tmp/dummy' # working directory
mask_fp = '/home/grg/spm/MNI_T1_brain_mask.nii' # explicit mask for the analysis
print 'Reading', excel_fp, 'looking up column', paramtype, '...'

a = mreg.multiple_regression_analysis(paramtype, excel_fp, analysis_wd, mask_fp, verbose=False)
#a.run()

Reading /home/grg/spm/data/designmatrix2.xls looking up column MD ...
